In [12]:
def read_steps(file_path):
    steps = []
    with open(file_path, 'r') as file:
        lines = file.readlines()[2:]  
        for line in lines:
            parts = line.strip().split('|')
            print(parts)
            if len(parts) >= 3:
                action = parts[2].strip()
                expected_result = parts[3].strip()
                steps.append((action, expected_result))
    return steps

In [13]:
m = read_steps("steps.txt")

['', ' 1 ', ' Log in to Digital Customer Portal as Admin ', ' Landing page is displayed ', '']
['', ' 2 ', ' Click "Admin" icon on top ribbon ', ' Page with 3 options for admins is displayed ', '']
['', ' 3 ', ' Verify that page contains following tiles:- User Management<br>- Escalation Criteria<br>- Admin Ruleset ', ' Page contains tiles:- User Management<br>- Escalation Criteria<br>- Admin Ruleset ', '']
['', ' 4 ', ' Click on tab "User Management" ', ' User management page is displayed with list of all users ', '']
['', ' 5 ', ' Enter GID, part of User name or e-mail in to search field ', ' Only users matching search conditions are displayed ', '']
['', ' 6 ', ' Click "Edit" button ', ' Popup with user details, assigned roles and teams is displayed.For reference check attached screenshot ', '']
['', ' 7 ', ' Click "Upate User" button ', ' User management page is dipslayed users matching search conditions ', '']


In [21]:
def read_java_code(file_path):
    with open(file_path, 'r') as file:
        return file.read()
    
# /Users/jitender/Documents/GitHub/siemens-code-gen/finetuning/test.ipynb
def extract_code_segments(java_code_path, steps):
    code_segments = []
    # Split the Java code into lines
    java_code = read_java_code(java_code_path)
    lines = java_code.splitlines()
    # Initialize a variable to keep track of the current line index
    current_line = 0
    capturing = False  # Flag to indicate if we are capturing lines

    for action, expected in steps:
        segment = []
        while current_line < len(lines):
            line = lines[current_line].strip()
            if line.startswith("//Step"):
                if capturing:  # If we were capturing, save the segment
                    code_segments.append("\n".join(segment).strip())
                    segment = []  # Clear the segment for the next step
                capturing = True  # Start capturing after the first step
            elif capturing:  # Only capture lines if we are in a capturing state
                segment.append(line)
            current_line += 1
        
        # Append the last segment if capturing was active
        if segment:
            code_segments.append("\n".join(segment).strip())

    return code_segments

In [22]:
extract_code_segments("output.java", m)

['tc.browser.start(WebDrv.EDGE, ETestData.QA_ENV_URL, new CoreStartOptions());\ntc.browser.localLogin();\nWaitFor.condition(() -> tc.tile.exists(ETile.REPORT_AN_ISSUE));\ntc.addStepInfo("Landing page is Displayed", true, tc.button.exists(EButton.CONTACT),\nnew ComparerOptions().takeScreenShotPlatform());',
 'WaitFor.condition(() -> tc.button.exists(EButton.ADMIN));\ntc.button.click(EButton.ADMIN);\nWaitFor.condition(() -> tc.tile.exists(ETile.USER_MANAGEMENT));\nList<String> actualTiles = tc.tile.getAllTilesName();\ntc.addStepInfo("Page with 3 options for admins is displayed", 3, actualTiles.size(),\nnew ComparerOptions().takeScreenShotPlatform());',
 'List<String> expectedTiles = Arrays.asList("User Management", "Escalation Criteria" ,"Admin Ruleset");\ntc.addStepInfo("""\nPage contains tiles:\n- User Management\n- Escalation Criteria\n- Admin Ruleset""", true, actualTiles.equals(expectedTiles), new ComparerOptions().takeScreenShotPlatform());',
 'tc.tile.open(ETile.USER_MANAGEMENT);\

In [3]:
import pandas as pd

In [49]:
data = pd.read_excel("/Users/jitender/Documents/GitHub/siemens-code-gen/DataSets/MDLA/MLDA_Internal_User_(1).xlsx")

In [50]:
data.head()

,ID,Work Item Type,Title,Test Step,Step Action,Step Expected,Area Path,Assigned To,State
0,865.0,Test Case,Internal user - Landing Page - Layout check,NaN,NaN,NaN,SHS IT Test Service\MDLA,Rahul R <rahul.r@siemens-healthineers.com>,Ready
1,NaN,NaN,NaN,1.0,Log in to Digital Customer Portal,Landing page is displayed,NaN,NaN,NaN
2,NaN,NaN,NaN,2.0,Check that page contains top ribbon with:-Siem...,Landing page top ribbon content is according s...,NaN,NaN,NaN
3,NaN,NaN,NaN,3.0,Check that page contains following tiles:- Rep...,Landing page tile content is according screens...,NaN,NaN,NaN
4,NaN,NaN,NaN,4.0,Click on tile Report an issue with an order or...,Page with details for reporting an issue is op...,NaN,NaN,NaN


In [51]:
output_data = {
    "id":[],
    "result":[]
}
for i,row in data.iterrows():
    if pd.notna(row["ID"]):
        output_data["id"].append(row["ID"])
        j=i+1
        result = []
        while j<len(data) and type(data["Step Action"][j]) == str :
            result.append((str(data["Step Action"][j]) , str(data["Step Expected"][j])))
            j+=1
        output_data["result"].append(result)
        



In [52]:
output_data

{'id': [865.0, 866.0, 867.0, 868.0, 869.0],
 'result': [[('Log in to Digital Customer Portal',
    'Landing page is displayed'),
   ('Check that page contains top ribbon with:-Siemens healthineers Logo\n-My Digital Lab Assistant (vX.X.X)\n-Contact icon\n-Language icon\n-Settings icon\n-Notification bell icon\n-Icon with name shortcut\n-Name of logged in user\n-Admin icon      \nFor reference check attached screenshot',
    'Landing page top ribbon content is according screenshot and consists of:- Contact icon\n- Settings icon\n- Notification bell icon\n- Icon with name shortcut\n- Name of logged in user\n- Admin icon'),
   ('Check that page contains following tiles:- Report an issue with an order or delivery\n- Show me my Requests\n- Question about an order or eSuport assistance\n- Question about my Account\n- Request Allocation or Saturday Delivery (SET Request)\n\nFor reference check attached screenshot',
    'Landing page tile content is according screenshot and consists of followin

In [53]:
len(output_data["id"])

5

In [54]:
len(output_data["result"])

5

In [55]:
for i in range(len(output_data["id"])):

    steps = output_data["result"][i]
    with open(f'/Users/jitender/Documents/GitHub/siemens-code-gen/finetuning/MDLA/Internal_User/TC_0{i+1}_{int(output_data["id"][i])}.txt', 'w') as file:
        # Write the table header
        file.write("| Step | Action | Expected Result |\n")
        file.write("|------|--------|------------------|\n")

        # Write each step
        for i, (action, expected_result) in enumerate(steps, start=1):
            # Replace newlines with HTML line breaks for formatting
            action = action.replace('\n', '<br>')
            expected_result = expected_result.replace('\n', '<br>')
            file.write(f"| {i} | {action} | {expected_result} |\n")

    print("Table written to steps.txt")

Table written to steps.txt
Table written to steps.txt
Table written to steps.txt
Table written to steps.txt
Table written to steps.txt
